In [1]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

Using TensorFlow backend.
C:\Users\Asad Shah\anaconda3\envs\AI2_course\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Asad Shah\anaconda3\envs\AI2_course\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Asad Shah\anaconda3\envs\AI2_course\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Asad Shah\a

In [2]:
df=pd.read_csv('C:/Users/Asad Shah/Documents/final/final/fer2013.csv')

In [3]:
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [3]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_features = 64
num_labels = 7
batch_size = 64
epochs = 65
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# print(f"shape:{X_train.shape}")
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

# model.summary()

#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


#Saving the  model to  use it later on
fer_json = model.to_json()
with open("newModel2.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("newModel2.h5")




Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 28709 samples, validate on 3589 samples
Epoch 1/65
28709/28709 [==============================] - 474s 17ms/step - loss: 1.7455 - acc: 0.2843 - val_loss: 1.6138 - val_acc: 0.3625
Epoch 2/65
28709/28709 [==============================] - 421s 15ms/step - loss: 1.5276 - acc: 0.4006 - val_loss: 1.4160 - val_acc: 0.4564
Epoch 3/65
28709/28709 [==============================] - 408s 14ms/step - loss: 1.4099 - acc: 0.4515 - val_loss: 1.3140 - val_acc: 0.4904
Epoch 4/65
28709/28709 [==============================] - 408s 14ms/step - loss: 1.3458 - acc: 0.4805 - val_loss: 1.2807 - val_acc: 0.5043
Epoch 5/65
28709/28709 [==============================] - 410s 14ms/step - loss: 1.3006 - acc: 0.4959 - val_loss: 1.2400 - val_acc: 0.5127
Epoch 6/65
28709/2870

28709/28709 [==============================] - 408s 14ms/step - loss: 0.6718 - acc: 0.7499 - val_loss: 1.3053 - val_acc: 0.5715
Epoch 53/65
28709/28709 [==============================] - 411s 14ms/step - loss: 0.6620 - acc: 0.7556 - val_loss: 1.3868 - val_acc: 0.5712
Epoch 54/65
28709/28709 [==============================] - 407s 14ms/step - loss: 0.6598 - acc: 0.7517 - val_loss: 1.3586 - val_acc: 0.5717
Epoch 55/65
28709/28709 [==============================] - 412s 14ms/step - loss: 0.6553 - acc: 0.7562 - val_loss: 1.3523 - val_acc: 0.5734
Epoch 56/65
28709/28709 [==============================] - 408s 14ms/step - loss: 0.6398 - acc: 0.7623 - val_loss: 1.4229 - val_acc: 0.5695
Epoch 57/65
28709/28709 [==============================] - 408s 14ms/step - loss: 0.6365 - acc: 0.7660 - val_loss: 1.3762 - val_acc: 0.5804
Epoch 58/65
28709/28709 [==============================] - 410s 14ms/step - loss: 0.6284 - acc: 0.7701 - val_loss: 1.3762 - val_acc: 0.5784
Epoch 59/65
28709/28709 [=======